# Basic usage

In [1]:
% add the relevant functions from the source and library folder
addpath(genpath(fullfile(pwd, '..', 'src')))
addpath(genpath(fullfile(pwd, '..', 'lib')))

## Setting the parameters about our experiment

We need to define the configuration structure where we put the information about the experiment.

In [2]:
% define the folder where the data will be saved
cfg.dir.output = fullfile(pwd, 'output');

cfg.verbose = 1;

% define the name of the task
cfg.task.name = 'test task';

% We can use the userInputs function to collect subject info: this won't work in a jupyter notebook though
% cfg = userInputs;

% so we can add those information manually directly
cfg.subject.subjectNb = 1;
cfg.subject.runNb = 1;

% if set to 'mri' then the data will be saved in the `func` folder
cfg.testingDevice = 'mri';

By default we assume you are running a purely behavioral experiment with no eyetracker.

```
cfg.eyeTracker = false;
cfg.testingDevice = 'pc';
```

If the testing device is set to `pc` then the data will be saved in the `beh` folder

If set to `eeg` then the data will be saved in the `eeg` folder

```
cfg.testingDevice = 'eeg';
```

In [3]:
% We can then initialize all the other fields. The ones we have already filled in will not be overwritten
cfg = checkCFG(cfg);



___________________________________________________
___________________________________________________
                                                   
         ___ ___ ___   ___ ___ ___  ___            
        / __| _ \ _ \ | _ )_ _|   \/ __|
       | (__|  _/  _/ | _ \| || |) \__ \
        \___|_| |_|   |___/___|___/|___/
                                                   
Thank you for using the CPP BIDS - version v1.0.0. 

Current list of contributors includes:
 Rémi Gau
 Marco Barilari
 Ceren Battal
 Tomas Lenc


Please cite using the following DOI: 
 https://doi.org/10.5281/zenodo.4007674

For bug report, suggestions or contributions see: 
 https://github.com/cpp-lln-lab/CPP_BIDS

___________________________________________________
___________________________________________________




## Creating BIDS paths and filenames

In [4]:
% create the filenames: this include a step to check that all the information is there: createFilename calls 
% checkCFG before running
cfg = createFilename(cfg);

### Output directory for this subject

Everything is put in a `source` folder to distinguish it from the `raw` that is usually reserved for the final 'true' BIDS data.

You can see that even though it was not specified a session folder and label is added to the folder structure and the filename.

In [5]:
disp(cfg.dir.outputSubject)

/home/remi/github/CPP_BIDS/notebooks/output/source/sub-001/ses-001


### filename for events.tsv files

You will note that we are converting the name of the task to a `camelCase`.

`test task` --> `testTask`

In [6]:
disp(cfg.fileName.events)

sub-001_ses-001_task-testTask_run-001_events_date-202011171229.tsv


The fullpath for this file would be:

In [7]:
disp(fullfile(...
    cfg.dir.outputSubject, ...
    cfg.fileName.modality, ...
    cfg.fileName.events))

/home/remi/github/CPP_BIDS/notebooks/output/source/sub-001/ses-001/func/sub-001_ses-001_task-testTask_run-001_events_date-202011171229.tsv


## Saving data to tsv file

### Setting things up

In [8]:
clear cfg logFile

cfg.dir.output = fullfile(pwd, 'output');

cfg.verbose = 0;

cfg.task.name = 'test task';

cfg.subject.subjectNb = 1;
cfg.subject.runNb = 1;

cfg.testingDevice = 'mri';

% Create the filenames
[cfg] = createFilename(cfg);

% We can define what extra columns we want in our tsv file beyond the 
% BIDS "holy trinity" ('onset', 'duration', 'trial_type')

% Say we want to keep track of the type of target that what presented during a trial and of its position
logFile.extraColumns = {'target_position', 'target_type'};


### Preparing your data dictionnary

We can then initialize our logfile.

In [9]:
logFile = saveEventsFile('init', cfg, logFile);

This will initialize a structure for each extra column.


In [10]:
disp(logFile.extraColumns.target_position.bids)


  scalar structure containing the fields:

    Description = 
    Levels = 
    LongName = 
    TermURL = 
    Units = 


You can then add information about each extraccolumns so they are saved in the JSON data dictionary that will go with the `.tsv` file.

In [11]:
logFile.extraColumns.target_position.bids.Description = 'Position of the target on the screen';
logFile.extraColumns.target_position.bids.Levels = {'left', 'center', 'right'};

disp(logFile.extraColumns.target_position.bids)


  scalar structure containing the fields:

    Description = Position of the target on the screen
    Levels = 
    {
      [1,1] = left
      [1,2] = center
      [1,3] = right
    }
    LongName = 
    TermURL = 
    Units = 


### Adding data during the experiment

A couple of things to do before you start the main loop of your experiment.

In [12]:
% create the output directory
mkdir(fullfile(cfg.dir.outputSubject, cfg.fileName.modality));
% open the file
logFile = saveEventsFile('open', cfg, logFile);

% This prints the header of the tsv file.


onset	duration	trial_type	target_position	target_type


Calling `tree` from the linux command line to see that we got what we want

In [13]:
!tree

.
├── basic_usage.ipynb
├── creating_BIDS_dataset.ipynb
├── more_on_saving.ipynb
├── output
│   └── source
│       └── sub-001
│           └── ses-001
│               └── func
│                   ├── sub-001_ses-001_task-testTask_run-001_events_date-202011171229.json
│                   └── sub-001_ses-001_task-testTask_run-001_events_date-202011171229.tsv
├── README.md
└── using_2_logfiles.ipynb

5 directories, 7 files



2 things to note.

- A `_date-YYYYMMDDHHMM` suffix is appended to the file name. This is NOT accepted by the BIDS filenaming convention. This was introduced to prevent overwriting files by mistake in case some things were re-run. When you convert the source data into raw data, this date suffix should be removed (the `removeDataSuffix` is there for that).

- A json file that stores your data dictionnary was created.

Let's inpsect its content.

In [14]:
!cat output/source/sub-001/ses-001/func/sub-001_ses-001_task-testTask_run-001_events_date-*.json

{
    "onset": {
        "Description": "time elapsed since experiment start",
        "Units": "s"
    },
    "trial_type": {
        "Description": "types of trial",
        "Levels": ""
    },
    "duration": {
        "Description": "duration of the event or the block",
        "Units": "s"
    },
    "target_position": {
        "Description": "Position of the target on the screen",
        "Levels": [
            "left",
            "center",
            "right"
        ],
        "LongName": "",
        "TermURL": "",
        "Units": ""
    },
    "target_type": {
        "Description": "",
        "Levels": "",
        "LongName": "",
        "TermURL": "",
        "Units": ""
    }
}


In [15]:
% The information about 2 events that we want to save
% NOTE : If the user DOES NOT provide `onset`, `duration`, this events will be skipped.

% Trial one
logFile(1,1).onset = 2;
logFile(1,1).trial_type = 'motion_up';
logFile(1,1).duration = 1;
logFile(1,1).target_position = 'left';
logFile(1,1).target_type = 'audio';

% Trial two
logFile(2,1).onset = 6;
logFile(2,1).trial_type = 'static';
logFile(2,1).duration = 1.2;
logFile(2,1).target_position = 'right';
logFile(2,1).target_type = 'visual';

% add those 2 events to the events.tsv file

logFile = saveEventsFile('save', cfg, logFile);


ans =  2
ans =  1

ans =  6
ans =  1.2000



The content of the `logfile` structure is emptied after every save so saving again will only throw a warning.

In [16]:
logFile = saveEventsFile('save', cfg, logFile);

 onset: NaN 
 duration: NaN 


Any missing information will be replaced by `n/a`.

And will throw a warning. They can be quite verbose on octave.

In [17]:
% New trial
logFile(1,1).onset = 10;
logFile(1,1).trial_type = 'motion_down';
logFile(1,1).duration = 1;
logFile(1,1).target_position = [];
logFile(1,1).target_type = 'audio';

logFile = saveEventsFile('save', cfg, logFile);

    saveEventsFile>checkExtracolumns at line 288 column 13
    saveEventsFile>checklLogFile at line 173 column 21
    saveEventsFile>saveToLogFile at line 351 column 17
    saveEventsFile at line 123 column 21
ans =  10
ans =  1



You can then close the file.

In [18]:
% close the file
saveEventsFile('close', cfg, logFile);

### Inspecting the output

The `bids-matlab` toolbox comes as part of CPP_BIDS as a submodule so you can easily load the content of this tsv file.

In [19]:
output = bids.util.tsvread( ...
    fullfile(...
    cfg.dir.outputSubject, ...
    cfg.fileName.modality, ...
    cfg.fileName.events))

output =

  scalar structure containing the fields:

    onset =

        2
        6
       10

    duration =

       1.0000
       1.2000
       1.0000

    trial_type = 
    {
      [1,1] = motion_up
      [2,1] = static
      [3,1] = motion_down
    }
    target_position = 
    {
      [1,1] = left
      [2,1] = right
      [3,1] = n/a
    }
    target_type = 
    {
      [1,1] = audio
      [2,1] = visual
      [3,1] = audio
    }

